In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('s3://ecc-project-dataset/hmnist_28_28_RGB.csv')

/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [3]:
data.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
0,192,153,193,195,155,192,197,154,185,202,...,173,124,138,183,147,166,185,154,177,2
1,25,14,30,68,48,75,123,93,126,158,...,60,39,55,25,14,28,25,14,27,2
2,192,138,153,200,145,163,201,142,160,206,...,167,129,143,159,124,142,136,104,117,2
3,38,19,30,95,59,72,143,103,119,171,...,44,26,36,25,12,17,25,12,15,2
4,158,113,139,194,144,174,215,162,191,225,...,209,166,185,172,135,149,109,78,92,2


In [4]:
df_target = data.iloc[:, -1]

In [5]:
df_target.shape

(10015,)

In [6]:
df_final = data.drop(data.columns[-1], axis=1)

In [7]:
df_final.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2342,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351
0,192,153,193,195,155,192,197,154,185,202,...,134,173,124,138,183,147,166,185,154,177
1,25,14,30,68,48,75,123,93,126,158,...,82,60,39,55,25,14,28,25,14,27
2,192,138,153,200,145,163,201,142,160,206,...,149,167,129,143,159,124,142,136,104,117
3,38,19,30,95,59,72,143,103,119,171,...,73,44,26,36,25,12,17,25,12,15
4,158,113,139,194,144,174,215,162,191,225,...,201,209,166,185,172,135,149,109,78,92


In [8]:
df_final.shape

(10015, 2352)

In [9]:
X = np.array(df_final)
y = np.array(df_target)

In [10]:
y = y.reshape(-1,1)
y.shape

(10015, 1)

In [11]:
 # spliting the data in to test and train sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5)

In [12]:
X_train

array([[119,  83, 111, ..., 138, 101, 125],
       [245, 168, 173, ..., 244, 171, 177],
       [201, 108, 123, ..., 188, 114, 115],
       ...,
       [207, 145, 127, ..., 207, 162, 135],
       [163, 161, 179, ..., 152, 132, 146],
       [247, 219, 231, ..., 248, 221, 222]])

In [13]:
!pip install xgboost

In [16]:
# Train an XGBoost regressor model 

import xgboost as xgb
model = xgb.XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            learning_rate=0.1, max_delta_step=0,
                                            max_depth=3, min_child_weight=1,
                                             n_estimators=100,
                                            n_jobs=-1, nthread=None,
                                            objective='binary:logistic',
                                            random_state=1855, reg_alpha=0,
                                            reg_lambda=1, scale_pos_weight=1,
                                            seed=None, silent=None, subsample=1,
                                            verbosity=0)

model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              device=None, early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=0, max_depth=3,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=-1, nthread=None, num_parallel_tree=None, ...)

In [17]:
y_pred = model.predict(X_test)

In [20]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.39      0.24      0.30        29
           1       0.58      0.33      0.42        45
           2       0.51      0.43      0.47        86
           3       0.00      0.00      0.00         5
           4       0.79      0.96      0.87       504
           5       1.00      0.10      0.18        10
           6       0.52      0.15      0.24        72

    accuracy                           0.74       751
   macro avg       0.54      0.32      0.35       751
weighted avg       0.70      0.74      0.70       751



/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

In [22]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.7376830892143809


In [ ]:
#train using sagemaker

In [23]:
# Convert the array into dataframe in a way that target variable is set as the first column and followed by feature columns
# This is because sagemaker built-in algorithm expects the data in this format.

train_data = pd.DataFrame({'Target': y_train[:,0]})
for i in range(X_train.shape[1]):
    train_data[i] = X_train[:,i]

/tmp/ipykernel_26018/2145295216.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data[i] = X_train[:,i]
/tmp/ipykernel_26018/2145295216.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data[i] = X_train[:,i]
/tmp/ipykernel_26018/2145295216.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.c

In [24]:
train_data.head()

,Target,0,1,2,3,4,5,6,7,8,...,2342,2343,2344,2345,2346,2347,2348,2349,2350,2351
0,4,119,83,111,133,101,131,142,115,149,...,153,154,128,154,148,119,147,138,101,125
1,4,245,168,173,244,170,177,248,175,182,...,182,246,177,181,245,174,180,244,171,177
2,4,201,108,123,205,113,126,206,111,121,...,112,190,107,114,190,111,113,188,114,115
3,0,180,145,163,183,146,163,185,152,169,...,172,185,155,166,175,140,148,143,104,116
4,4,160,132,125,168,142,139,177,153,155,...,155,170,143,151,163,133,140,156,126,128


In [25]:
val_data = pd.DataFrame({'Target':y_val[:,0]})
for i in range(X_val.shape[1]):
    val_data[i] = X_val[:,i]

/tmp/ipykernel_26018/565076207.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  val_data[i] = X_val[:,i]
/tmp/ipykernel_26018/565076207.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  val_data[i] = X_val[:,i]
/tmp/ipykernel_26018/565076207.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  va

In [26]:
val_data.head()

,Target,0,1,2,3,4,5,6,7,8,...,2342,2343,2344,2345,2346,2347,2348,2349,2350,2351
0,2,206,159,167,209,163,173,209,161,173,...,163,205,149,165,204,141,155,202,139,151
1,4,62,50,48,102,86,80,130,115,105,...,129,120,114,111,82,75,77,41,37,41
2,4,155,125,132,159,131,138,162,133,140,...,106,142,103,100,127,94,91,88,64,62
3,4,243,148,135,246,153,143,249,152,143,...,127,223,134,118,220,133,117,219,136,116
4,4,193,180,211,190,175,204,194,181,209,...,186,186,168,188,187,171,192,182,166,189


In [27]:
val_data.shape

(752, 2353)

In [28]:
# save train_data and validation_data as csv files.

train_data.to_csv('train.csv', header = False, index = False)
val_data.to_csv('validation.csv', header = False, index = False)
df_final.to_csv('full_data.csv', index=False)

In [30]:
pip install sagemaker==2.182.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 4.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.182.0-py2.py3-none-any.whl size=1185307 sha256=b98a58e78a52ac0e6e3e6d9a1979cd5a89c824f9e67021bc287a7b6b2618bb4d
  Stored in directory: /home/ec2-user/.cache/pip/wheels/07/0c/af/ead4148322733a12bce80af91e36ecb67da78cc84fc01555bf
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.199.0
    Uninstalling sagemaker-2.199.0:
      Successfully uninstalled sagemaker-2.199.0
Note: you may need to restart the kernel to use updated packages.


In [33]:
# Boto3 is the Amazon Web Services (AWS) Software Development Kit (SDK) for Python
# Boto3 allows Python developer to write software that makes use of services like Amazon S3 and Amazon EC2

import sagemaker
import boto3

# Create a sagemaker session
sagemaker_session = sagemaker.Session()

#S 3 bucket and prefix that we want to use
# default_bucket - csreates a Amazon S3 bucket to be used in this session
bucket = 'ecc-project-dataset'
prefix = 'XGBoost-Classifier_new'
key = 'XGBoost-Classifier_new'
#Roles give learning and hosting access to the data
#This is specified while opening the sagemakers instance in "Create an IAM role"
role = sagemaker.get_execution_role()

In [34]:
print(role)

arn:aws:iam::087733582531:role/service-role/AmazonSageMaker-ExecutionRole-20231025T233248


In [35]:
# read the data from csv file and then upload the data to s3 bucket
import os
with open('train.csv','rb') as f:
    # The following code uploads the data into S3 bucket to be accessed later for training
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(f)

In [36]:
# Let's print out the training data location in s3
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('Uploaded the Training Data at location: {}'.format(s3_train_data))

Uploaded the Training Data at location: s3://ecc-project-dataset/XGBoost-Classifier_new/train/XGBoost-Classifier_new


In [37]:
# read the data from csv file and then upload the data to s3 bucket

with open('validation.csv','rb') as f:
    # The following code uploads the data into S3 bucket to be accessed later for training

    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation', key)).upload_fileobj(f)

In [38]:
# Let's print out the validation data location in s3
s3_validation_data = 's3://{}/{}/validation/{}'.format(bucket, prefix, key)
print('uploaded validation data location: {}'.format(s3_validation_data))

uploaded validation data location: s3://ecc-project-dataset/XGBoost-Classifier_new/validation/XGBoost-Classifier_new


In [40]:
# creates output placeholder in S3 bucket to store the output

output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))

training artifacts will be uploaded to: s3://ecc-project-dataset/XGBoost-Classifier_new/output


In [41]:
# This code is used to get the training container of sagemaker built-in algorithms
# all we have to do is to specify the name of the algorithm, that we want to use

# Let's obtain a reference to the XGBoost container image
# Note that all regression models are named estimators
# You don't have to specify (hardcode) the region, get_image_uri will get the current region name using boto3.Session

from sagemaker.image_uris import retrieve

container = retrieve(region=boto3.Session().region_name, framework='xgboost',version='latest') # Latest

In [53]:
# Specify the type of instance that we would like to use for training 
# output path and sagemaker session into the Estimator. 
# We can also specify how many instances we would like to use for training

# Recall that XGBoost works by combining an ensemble of weak models to generate accurate/robust results. 
# The weak models are randomized to avoid overfitting

# num_round: The number of rounds to run the training.


# Alpha: L1 regularization term on weights. Increasing this value makes models more conservative.

# colsample_by_tree: fraction of features that will be used to train each tree.

# eta: Step size shrinkage used in updates to prevent overfitting. 
# After each boosting step, eta parameter shrinks the feature weights to make the boosting process more conservative.


Xgboost_classifier1 = sagemaker.estimator.Estimator(container,
                                       role, 
                                       instance_count = 1, 
                                       instance_type = 'ml.m5.xlarge',
                                       output_path = output_location,
                                       sagemaker_session = sagemaker_session,content_type='csv')

#We can tune the hyper-parameters to improve the performance of the model

Xgboost_classifier1.set_hyperparameters(max_depth=3,
                        base_score=0.5, booster='gbtree',
                         min_child_weight=1,   num_round=100,
                                         n_estimators=100,
                                            n_jobs=-1,random_state=1855, reg_alpha=0,
                                            reg_lambda=1, scale_pos_weight=1,
                                           subsample=1,
                                            verbosity=0
                           )


In [54]:
# Creating "train", "validation" channels to feed in the model
# Source: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html

train_input = sagemaker.inputs.TrainingInput(s3_data = s3_train_data, content_type='csv',s3_data_type = 'S3Prefix')
valid_input = sagemaker.inputs.TrainingInput(s3_data = s3_validation_data, content_type='csv',s3_data_type = 'S3Prefix')


data_channels = {'train': train_input,'validation': valid_input}


Xgboost_classifier1.fit(data_channels)

INFO:sagemaker:Creating training-job with name: xgboost-2023-12-06-15-32-02-252


2023-12-06 15:32:02 Starting - Starting the training job...
2023-12-06 15:32:16 Starting - Preparing the instances for training......
2023-12-06 15:33:21 Downloading - Downloading input data...
2023-12-06 15:34:01 Training - Downloading the training image...
2023-12-06 15:34:17 Training - Training image download completed. Training in progress.Arguments: train
[2023-12-06:15:34:26:INFO] Running standalone xgboost training.
[2023-12-06:15:34:26:INFO] File size need to be processed in the node: 80.98mb. Available memory size in the node: 8221.86mb
[2023-12-06:15:34:26:INFO] Determined delimiter of CSV input is ','
[15:34:26] S3DistributionType set as FullyReplicated
[15:34:26] 8512x2352 matrix with 20020224 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2023-12-06:15:34:26:INFO] Determined delimiter of CSV input is ','
[15:34:26] S3DistributionType set as FullyReplicated
[15:34:26] 752x2352 matrix with 1768704 entries loaded from /opt/ml/input/data/va

In [55]:
# Deploy the model to perform inference 
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

Xgboost_classifier = Xgboost_classifier1.deploy(initial_instance_count = 1, instance_type = 'ml.m5.xlarge',serializer = CSVSerializer('text/csv'))

INFO:sagemaker:Creating model with name: xgboost-2023-12-06-15-37-49-960
INFO:sagemaker:Creating endpoint-config with name xgboost-2023-12-06-15-37-49-960
INFO:sagemaker:Creating endpoint with name xgboost-2023-12-06-15-37-49-960


----!

In [ ]:
pred = Xgboost_classifier.predict